# Dynamic Graph Anomaly Detection - Comprehensive Evaluation

This notebook provides a comprehensive evaluation framework for the DiGress-based anomaly detection system. It covers:

1. **Model Performance Evaluation**
2. **Reconstruction Error Analysis** 
3. **Threshold Selection & Validation**
4. **Temporal Feature Importance**
5. **Attention Mechanism Analysis**
6. **Anomaly Detection Metrics**
7. **Comparative Analysis**
8. **Visualization & Interpretation**

In [1]:
# Import necessary libraries
import sys
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix
from sklearn.manifold import TSNE
from scipy import stats
import networkx as nx
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

# Fix project paths - we need to go up to the project root
project_root = os.path.dirname(os.path.dirname(os.getcwd()))  # Go up from src/metrics to project root
src_path = os.path.join(project_root, 'src')
anomalies_path = os.path.join(project_root, 'anomalies_detection')

sys.path.insert(0, project_root)
sys.path.insert(0, src_path)
sys.path.insert(0, anomalies_path)

print(f"Project root: {project_root}")
print(f"Source path: {src_path}")
print(f"Anomalies path: {anomalies_path}")

# Import project modules with corrected paths
try:
    from models.denoising_network import DenoisingNetwork
    from models.transition_matrices import DiGressTransitionMatrices
    from models.graph_features import compute_graph_features
    from models.graph_temporel_features import compute_temporal_graph_features
    from models.visualize_snapshot import SnapshotLoader, get_snapshot_files
    print("✅ Core models imported successfully!")
except ImportError as e:
    print(f"❌ Error importing core models: {e}")

try:
    from run_anomaly_detection import (
        load_trained_model_and_threshold, 
        compute_single_graph_error,
        detect_anomaly,
        detect_multiple_anomalies
    )
    print("✅ Anomaly detection functions imported successfully!")
except ImportError as e:
    print(f"❌ Error importing anomaly detection functions: {e}")

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Evaluation framework loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

Project root: /home/sofien/Desktop/anomalies_detections
Source path: /home/sofien/Desktop/anomalies_detections/src
Anomalies path: /home/sofien/Desktop/anomalies_detections/anomalies_detection
✅ Core models imported successfully!
✅ Anomaly detection functions imported successfully!
📊 Evaluation framework loaded successfully!
PyTorch version: 2.7.0+cu126
Device available: cpu


## 1. Configuration & Data Loading

In [2]:
# Configuration parameters
CONFIG = {
    'dataset': 'uci',
    'mode': 'dynamic',
    'device': torch.device('cpu'),
    'num_test_samples': 100,
    'random_seed': 42,
    'model_path': os.path.join(project_root, 'saved_models', 'digress_dynamic_final_uci.pt'),
    'threshold_path': os.path.join(project_root, 'saved_models', 'threshold_vector_dynamic_uci.pt')
}

# Set random seed for reproducibility
torch.manual_seed(CONFIG['random_seed'])
np.random.seed(CONFIG['random_seed'])

print(f"📁 Configuration:")
print(f"   Dataset: {CONFIG['dataset']}")
print(f"   Mode: {CONFIG['mode']}")
print(f"   Model path: {CONFIG['model_path']}")
print(f"   Threshold path: {CONFIG['threshold_path']}")

# Load available snapshots
try:
    all_snapshots = get_snapshot_files(CONFIG['dataset'])
    print(f"📁 Total snapshots available: {len(all_snapshots)}")
    
    # Sample for evaluation (last 20% as test set)
    test_size = int(len(all_snapshots) * 0.2)
    test_snapshots = all_snapshots[-test_size:]
    train_snapshots = all_snapshots[:-test_size]

    print(f"🎯 Test snapshots: {len(test_snapshots)}")
    print(f"🏋️ Train snapshots: {len(train_snapshots)}")

    # Display sample snapshot info
    if test_snapshots:
        sample_snapshot = test_snapshots[0]
        X_sample = SnapshotLoader.get_X(sample_snapshot)
        E_sample = SnapshotLoader.get_E(sample_snapshot)
        print(f"\n📊 Sample snapshot dimensions:")
        print(f"   X (nodes): {X_sample.shape}")
        print(f"   E (edges): {E_sample.shape}")
    else:
        print("⚠️ No test snapshots found")
        
except Exception as e:
    print(f"❌ Error loading snapshots: {e}")
    print("Please ensure you have generated snapshots by running the data preparation scripts first.")

📁 Configuration:
   Dataset: uci
   Mode: dynamic
   Model path: /home/sofien/Desktop/anomalies_detections/saved_models/digress_dynamic_final_uci.pt
   Threshold path: /home/sofien/Desktop/anomalies_detections/saved_models/threshold_vector_dynamic_uci.pt
📁 Total snapshots available: 0
🎯 Test snapshots: 0
🏋️ Train snapshots: 0
⚠️ No test snapshots found


## 2. Model Performance Evaluation

In [5]:
# Load trained model and threshold vector
try:
    model, transition_matrices, threshold_vector = load_trained_model_and_threshold(
        dataset=CONFIG['dataset'], 
        mode=CONFIG['mode']
    )
    print("✅ Model and threshold loaded successfully!")
    
    # Display model information
    print(f"\n🔧 Model Configuration:")
    print(f"   Mode: {CONFIG['mode']}")
    print(f"   Device: {CONFIG['device']}")
    
    # Display threshold information
    print(f"\n📊 Threshold Vector:")
    for key, value in threshold_vector.items():
        if isinstance(value, (int, float)):
            print(f"   {key}: {value:.6f}")
        else:
            print(f"   {key}: {value}")
            
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please ensure you have run main.py to train the model first!")

 Chargement du modèle et seuil pour uci (mode: dynamic)
❌ Error loading model:  Modèle non trouvé: saved_models/digress_dynamic_final_uci.pt
 Exécutez d'abord main.py pour entraîner le modèle
Please ensure you have run main.py to train the model first!


## 3. Reconstruction Error Analysis

In [4]:
# Compute reconstruction errors for test set
print("🔄 Computing reconstruction errors for test set...")

reconstruction_errors = []
test_samples = test_snapshots[:CONFIG['num_test_samples']]  # Limit for faster evaluation

for i, snapshot_path in enumerate(test_samples):
    try:
        # Load snapshot
        X = SnapshotLoader.get_X(snapshot_path)
        E = SnapshotLoader.get_E(snapshot_path)
        
        # Compute reconstruction error
        error = compute_single_graph_error(X, E, model, transition_matrices, CONFIG['mode'])
        reconstruction_errors.append(error)
        
        if (i + 1) % 20 == 0:
            print(f"   Processed {i+1}/{len(test_samples)} samples...")
            
    except Exception as e:
        print(f"   ⚠️ Error processing sample {i}: {e}")
        continue

reconstruction_errors = np.array(reconstruction_errors)
print(f"✅ Computed {len(reconstruction_errors)} reconstruction errors")

# Basic statistics
print(f"\n📊 Reconstruction Error Statistics:")
print(f"   Mean: {np.mean(reconstruction_errors):.6f}")
print(f"   Std: {np.std(reconstruction_errors):.6f}")
print(f"   Min: {np.min(reconstruction_errors):.6f}")
print(f"   Max: {np.max(reconstruction_errors):.6f}")
print(f"   Median: {np.median(reconstruction_errors):.6f}")

# Percentiles
percentiles = [50, 75, 90, 95, 99]
print(f"\n📈 Percentiles:")
for p in percentiles:
    value = np.percentile(reconstruction_errors, p)
    print(f"   {p}th percentile: {value:.6f}")

🔄 Computing reconstruction errors for test set...
✅ Computed 0 reconstruction errors

📊 Reconstruction Error Statistics:
   Mean: nan
   Std: nan


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
# Visualize reconstruction error distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram
axes[0, 0].hist(reconstruction_errors, bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].axvline(threshold_vector['percentile_95_threshold'], color='red', linestyle='--', 
                   label=f"95% Threshold: {threshold_vector['percentile_95_threshold']:.4f}")
axes[0, 0].axvline(np.mean(reconstruction_errors), color='green', linestyle='--', 
                   label=f"Mean: {np.mean(reconstruction_errors):.4f}")
axes[0, 0].set_title('Reconstruction Error Distribution')
axes[0, 0].set_xlabel('Reconstruction Error')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Box plot
axes[0, 1].boxplot(reconstruction_errors, vert=True)
axes[0, 1].set_title('Reconstruction Error Box Plot')
axes[0, 1].set_ylabel('Reconstruction Error')
axes[0, 1].grid(True, alpha=0.3)

# Q-Q plot (normality test)
stats.probplot(reconstruction_errors, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (Normality Test)')
axes[1, 0].grid(True, alpha=0.3)

# Log-scale distribution
axes[1, 1].hist(np.log1p(reconstruction_errors), bins=50, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Log-Scale Error Distribution')
axes[1, 1].set_xlabel('Log(1 + Reconstruction Error)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical tests
shapiro_stat, shapiro_p = stats.shapiro(reconstruction_errors[:min(5000, len(reconstruction_errors))])
print(f"\n🧪 Statistical Tests:")
print(f"   Shapiro-Wilk test (normality): statistic={shapiro_stat:.4f}, p-value={shapiro_p:.6f}")
print(f"   Distribution appears {'normal' if shapiro_p > 0.05 else 'non-normal'} (α=0.05)")

## 4. Threshold Selection & Validation

In [ ]:
# Analyze different threshold strategies
threshold_strategies = {
    'percentile_95': np.percentile(reconstruction_errors, 95),
    'percentile_99': np.percentile(reconstruction_errors, 99),
    'mean_plus_2std': np.mean(reconstruction_errors) + 2 * np.std(reconstruction_errors),
    'mean_plus_3std': np.mean(reconstruction_errors) + 3 * np.std(reconstruction_errors),
    'iqr_outlier': np.percentile(reconstruction_errors, 75) + 1.5 * (np.percentile(reconstruction_errors, 75) - np.percentile(reconstruction_errors, 25))
}

# Compare with stored thresholds
print("🎯 Threshold Comparison:")
print(f"{'Strategy':<20} {'Computed':<12} {'Stored':<12} {'Difference':<12}")
print("-" * 60)

for strategy, computed_value in threshold_strategies.items():
    stored_key = f"{strategy}_threshold"
    stored_value = threshold_vector.get(stored_key, "N/A")
    
    if isinstance(stored_value, (int, float)):
        diff = abs(computed_value - stored_value)
        print(f"{strategy:<20} {computed_value:<12.6f} {stored_value:<12.6f} {diff:<12.6f}")
    else:
        print(f"{strategy:<20} {computed_value:<12.6f} {'N/A':<12} {'N/A':<12}")

# Analyze anomaly detection rates for different thresholds
print(f"\n📊 Anomaly Detection Rates:")
print(f"{'Threshold':<20} {'Value':<12} {'Anomaly Rate':<12} {'Count':<8}")
print("-" * 55)

for name, threshold_val in threshold_strategies.items():
    anomalies = reconstruction_errors > threshold_val
    anomaly_rate = np.mean(anomalies)
    anomaly_count = np.sum(anomalies)
    print(f"{name:<20} {threshold_val:<12.6f} {anomaly_rate:<12.3%} {anomaly_count:<8}")

# ROC-like analysis (assuming last 10% as anomalies for evaluation)
n_true_anomalies = int(len(reconstruction_errors) * 0.1)
true_anomaly_indices = np.argsort(reconstruction_errors)[-n_true_anomalies:]
true_labels = np.zeros(len(reconstruction_errors))
true_labels[true_anomaly_indices] = 1

# Compute metrics for different thresholds
thresholds_to_test = np.percentile(reconstruction_errors, np.linspace(50, 99.9, 50))
precision_scores = []
recall_scores = []
f1_scores = []

for thresh in thresholds_to_test:
    predicted = reconstruction_errors > thresh
    
    if np.sum(predicted) > 0:
        precision = np.sum(predicted & (true_labels == 1)) / np.sum(predicted)
        recall = np.sum(predicted & (true_labels == 1)) / np.sum(true_labels)
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    else:
        precision = recall = f1 = 0
    
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Find optimal threshold
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds_to_test[optimal_idx]
optimal_f1 = f1_scores[optimal_idx]

print(f"\n🎯 Optimal Threshold Analysis (assuming top 10% as anomalies):")
print(f"   Optimal threshold: {optimal_threshold:.6f}")
print(f"   Optimal F1-score: {optimal_f1:.4f}")
print(f"   Precision at optimal: {precision_scores[optimal_idx]:.4f}")
print(f"   Recall at optimal: {recall_scores[optimal_idx]:.4f}")

## 5. Temporal Feature Importance Analysis

In [ ]:
# Extract and analyze temporal features from a sample of graphs
print("🔍 Analyzing temporal features...")

# Feature names corresponding to the 22 temporal features
TEMPORAL_FEATURE_NAMES = [
    't_norm', 'seq_position', 'edge_birth_rate', 'edge_death_rate', 'edge_stability',
    'edge_intermittency', 'degree_evolution_rate', 'degree_evolution_volatility', 
    'density_evolution', 'neighborhood_stability', 'clustering_evolution',
    'triangle_evolution', 'structural_autocorr', 'structural_periodicity',
    'centrality_evolution_mean', 'centrality_evolution_variance', 'centrality_persistence',
    'structural_entropy', 'structural_surprise', 'edge_predictability',
    'global_volatility', 'temporal_trend'
]

# Collect temporal features from sample snapshots
temporal_features_matrix = []
sample_size = min(50, len(test_samples))  # Use subset for faster computation

for i in range(sample_size):
    try:
        snapshot_path = test_samples[i]
        X = SnapshotLoader.get_X(snapshot_path)
        E = SnapshotLoader.get_E(snapshot_path)
        
        # Create a simple graph sequence for temporal feature computation
        class SimpleGraph:
            def __init__(self, X, E):
                self.X = X
                self.E = E
        
        graph = SimpleGraph(X, E)
        
        # Compute temporal features
        features = compute_temporal_graph_features(
            [graph] * 3,  # Simple sequence
            current_t=500,  # Middle timestep
            num_timesteps=1000,
            device=CONFIG['device'],
            window_size=3
        )
        
        temporal_features_matrix.append(features.cpu().numpy())
        
        if (i + 1) % 10 == 0:
            print(f"   Extracted features from {i+1}/{sample_size} samples...")
            
    except Exception as e:
        print(f"   ⚠️ Error extracting features from sample {i}: {e}")
        continue

temporal_features_matrix = np.array(temporal_features_matrix)
print(f"✅ Extracted temporal features: {temporal_features_matrix.shape}")

# Analyze feature statistics
feature_stats = pd.DataFrame({
    'Feature': TEMPORAL_FEATURE_NAMES[:temporal_features_matrix.shape[1]],
    'Mean': np.mean(temporal_features_matrix, axis=0),
    'Std': np.std(temporal_features_matrix, axis=0),
    'Min': np.min(temporal_features_matrix, axis=0),
    'Max': np.max(temporal_features_matrix, axis=0),
    'Variance': np.var(temporal_features_matrix, axis=0)
})

print(f"\n📊 Temporal Feature Statistics:")
print(feature_stats.round(4))

# Feature correlation analysis
feature_corr = np.corrcoef(temporal_features_matrix.T)
feature_corr_df = pd.DataFrame(
    feature_corr, 
    index=TEMPORAL_FEATURE_NAMES[:temporal_features_matrix.shape[1]],
    columns=TEMPORAL_FEATURE_NAMES[:temporal_features_matrix.shape[1]]
)

print(f"\n🔗 Highly Correlated Feature Pairs (|r| > 0.7):")
high_corr_pairs = []
for i in range(len(feature_corr_df)):
    for j in range(i+1, len(feature_corr_df)):
        corr_val = feature_corr_df.iloc[i, j]
        if abs(corr_val) > 0.7:
            high_corr_pairs.append((
                feature_corr_df.index[i], 
                feature_corr_df.columns[j], 
                corr_val
            ))

for feat1, feat2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
    print(f"   {feat1} ↔ {feat2}: {corr:.3f}")

In [ ]:
# Visualize temporal features
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Feature importance (variance-based)
feature_importance = feature_stats['Variance'].values
sorted_indices = np.argsort(feature_importance)[::-1]

axes[0, 0].barh(range(len(feature_importance)), feature_importance[sorted_indices])
axes[0, 0].set_yticks(range(len(feature_importance)))
axes[0, 0].set_yticklabels([TEMPORAL_FEATURE_NAMES[i] for i in sorted_indices], fontsize=8)
axes[0, 0].set_xlabel('Variance (Feature Importance)')
axes[0, 0].set_title('Temporal Feature Importance (Variance-based)')
axes[0, 0].grid(True, alpha=0.3)

# Feature correlation heatmap (top 10 features)
top_features_idx = sorted_indices[:10]
top_features_corr = feature_corr[np.ix_(top_features_idx, top_features_idx)]
top_features_names = [TEMPORAL_FEATURE_NAMES[i] for i in top_features_idx]

im = axes[0, 1].imshow(top_features_corr, cmap='coolwarm', vmin=-1, vmax=1)
axes[0, 1].set_xticks(range(len(top_features_names)))
axes[0, 1].set_yticks(range(len(top_features_names)))
axes[0, 1].set_xticklabels(top_features_names, rotation=45, ha='right', fontsize=8)
axes[0, 1].set_yticklabels(top_features_names, fontsize=8)
axes[0, 1].set_title('Feature Correlation Matrix (Top 10)')
plt.colorbar(im, ax=axes[0, 1])

# Feature distribution (top 5 most important)
top_5_features = temporal_features_matrix[:, sorted_indices[:5]]
axes[1, 0].boxplot(top_5_features, labels=[TEMPORAL_FEATURE_NAMES[i] for i in sorted_indices[:5]])
axes[1, 0].set_title('Distribution of Top 5 Features')
axes[1, 0].set_ylabel('Feature Value')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# PCA analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
features_pca = pca.fit_transform(temporal_features_matrix)

axes[1, 1].scatter(features_pca[:, 0], features_pca[:, 1], alpha=0.6)
axes[1, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[1, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[1, 1].set_title('PCA of Temporal Features')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🔍 PCA Analysis:")
print(f"   PC1 explains {pca.explained_variance_ratio_[0]:.1%} of variance")
print(f"   PC2 explains {pca.explained_variance_ratio_[1]:.1%} of variance")
print(f"   Total explained: {sum(pca.explained_variance_ratio_):.1%}")

# Top contributing features to each PC
feature_contributions_pc1 = pca.components_[0]
feature_contributions_pc2 = pca.components_[1]

print(f"\n📊 Top Contributing Features:")
print("PC1 (top 5):")
pc1_top = np.argsort(np.abs(feature_contributions_pc1))[-5:][::-1]
for idx in pc1_top:
    contrib = feature_contributions_pc1[idx]
    print(f"   {TEMPORAL_FEATURE_NAMES[idx]}: {contrib:.3f}")

print("PC2 (top 5):")
pc2_top = np.argsort(np.abs(feature_contributions_pc2))[-5:][::-1]
for idx in pc2_top:
    contrib = feature_contributions_pc2[idx]
    print(f"   {TEMPORAL_FEATURE_NAMES[idx]}: {contrib:.3f}")

## 6. Anomaly Detection Performance Metrics

In [ ]:
# Comprehensive anomaly detection evaluation
print("🎯 Evaluating Anomaly Detection Performance...")

# Create synthetic anomalies by adding noise to normal graphs
def create_synthetic_anomaly(X, E, noise_level=0.3):
    """Create synthetic anomaly by adding noise to graph"""
    X_anomaly = X.clone()
    E_anomaly = E.clone()
    
    # Add noise to node features
    if X.dtype == torch.float:
        noise = torch.randn_like(X) * noise_level
        X_anomaly = X + noise
    else:
        # For categorical data, randomly flip some categories
        mask = torch.rand_like(X.float()) < noise_level
        X_anomaly = torch.where(mask, 1 - X, X)
    
    # Add noise to edge features
    if E.dtype == torch.float:
        noise = torch.randn_like(E) * noise_level
        E_anomaly = E + noise
    else:
        # Randomly flip some edges
        mask = torch.rand_like(E.float()) < noise_level
        E_anomaly = torch.where(mask, 1 - E, E)
    
    return X_anomaly, E_anomaly

# Generate test set with known labels
normal_samples = test_samples[:25]  # First 25 as normal
anomaly_samples = []
true_labels = []

# Normal samples
normal_errors = []
for snapshot_path in normal_samples:
    X = SnapshotLoader.get_X(snapshot_path)
    E = SnapshotLoader.get_E(snapshot_path)
    error = compute_single_graph_error(X, E, model, transition_matrices, CONFIG['mode'])
    normal_errors.append(error)
    true_labels.append(0)  # Normal

# Synthetic anomaly samples
anomaly_errors = []
for snapshot_path in normal_samples:  # Use same graphs but add noise
    X = SnapshotLoader.get_X(snapshot_path)
    E = SnapshotLoader.get_E(snapshot_path)
    
    # Create anomaly version
    X_anom, E_anom = create_synthetic_anomaly(X, E, noise_level=0.2)
    error = compute_single_graph_error(X_anom, E_anom, model, transition_matrices, CONFIG['mode'])
    anomaly_errors.append(error)
    true_labels.append(1)  # Anomaly

all_errors = normal_errors + anomaly_errors
true_labels = np.array(true_labels)

print(f"📊 Test Set Composition:")
print(f"   Normal samples: {len(normal_errors)}")
print(f"   Anomaly samples: {len(anomaly_errors)}")
print(f"   Total samples: {len(all_errors)}")

# Evaluate performance with different thresholds
thresholds_to_evaluate = [
    ('percentile_95', threshold_vector['percentile_95_threshold']),
    ('percentile_99', threshold_vector['percentile_99_threshold']),
    ('z_score_2', threshold_vector['z_score_threshold_2']),
    ('optimal', optimal_threshold)
]

performance_results = {}

for thresh_name, thresh_value in thresholds_to_evaluate:
    predicted_labels = (np.array(all_errors) > thresh_value).astype(int)
    
    # Calculate metrics
    tn = np.sum((predicted_labels == 0) & (true_labels == 0))
    fp = np.sum((predicted_labels == 1) & (true_labels == 0))
    fn = np.sum((predicted_labels == 0) & (true_labels == 1))
    tp = np.sum((predicted_labels == 1) & (true_labels == 1))
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    performance_results[thresh_name] = {
        'threshold': thresh_value,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'f1_score': f1,
        'confusion_matrix': [[tn, fp], [fn, tp]]
    }

# Display results
print(f"\n🏆 Performance Results:")
print(f"{'Threshold':<15} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Specificity':<12}")
print("-" * 80)

for thresh_name, metrics in performance_results.items():
    print(f"{thresh_name:<15} {metrics['accuracy']:<10.3f} {metrics['precision']:<10.3f} "
          f"{metrics['recall']:<10.3f} {metrics['f1_score']:<10.3f} {metrics['specificity']:<12.3f}")

# Find best performing threshold
best_threshold = max(performance_results.keys(), key=lambda k: performance_results[k]['f1_score'])
print(f"\n🥇 Best performing threshold: {best_threshold} (F1: {performance_results[best_threshold]['f1_score']:.3f})")

# ROC Curve analysis
if len(set(true_labels)) > 1:  # Ensure we have both classes
    try:
        auc_score = roc_auc_score(true_labels, all_errors)
        print(f"📈 AUC-ROC Score: {auc_score:.3f}")
    except Exception as e:
        print(f"⚠️ Could not compute AUC-ROC: {e}")

# Error distribution by class
print(f"\n📊 Error Distribution by Class:")
print(f"   Normal samples - Mean: {np.mean(normal_errors):.6f}, Std: {np.std(normal_errors):.6f}")
print(f"   Anomaly samples - Mean: {np.mean(anomaly_errors):.6f}, Std: {np.std(anomaly_errors):.6f}")
print(f"   Separation ratio: {np.mean(anomaly_errors) / np.mean(normal_errors):.2f}x")

In [ ]:
# Visualize anomaly detection results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Error distribution by class
axes[0, 0].hist(normal_errors, bins=20, alpha=0.7, label='Normal', color='blue', density=True)
axes[0, 0].hist(anomaly_errors, bins=20, alpha=0.7, label='Anomaly', color='red', density=True)
axes[0, 0].axvline(performance_results['percentile_95']['threshold'], 
                   color='green', linestyle='--', label='95% Threshold')
axes[0, 0].set_xlabel('Reconstruction Error')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Error Distribution by Class')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Confusion matrix for best threshold
best_cm = performance_results[best_threshold]['confusion_matrix']
im = axes[0, 1].imshow(best_cm, interpolation='nearest', cmap='Blues')
axes[0, 1].set_title(f'Confusion Matrix ({best_threshold})')
axes[0, 1].set_ylabel('True Label')
axes[0, 1].set_xlabel('Predicted Label')

# Add text annotations
for i in range(2):
    for j in range(2):
        axes[0, 1].text(j, i, str(best_cm[i][j]), 
                        ha="center", va="center", fontsize=12)

axes[0, 1].set_xticks([0, 1])
axes[0, 1].set_yticks([0, 1])
axes[0, 1].set_xticklabels(['Normal', 'Anomaly'])
axes[0, 1].set_yticklabels(['Normal', 'Anomaly'])

# Performance comparison across thresholds
metrics_names = ['accuracy', 'precision', 'recall', 'f1_score', 'specificity']
threshold_names = list(performance_results.keys())

x = np.arange(len(threshold_names))
width = 0.15

for i, metric in enumerate(metrics_names):
    values = [performance_results[t][metric] for t in threshold_names]
    axes[1, 0].bar(x + i*width, values, width, label=metric.replace('_', ' ').title())

axes[1, 0].set_xlabel('Threshold Strategy')
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_title('Performance Metrics Comparison')
axes[1, 0].set_xticks(x + width * 2)
axes[1, 0].set_xticklabels(threshold_names, rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# ROC-like curve
fpr_values = []
tpr_values = []
threshold_range = np.linspace(min(all_errors), max(all_errors), 100)

for thresh in threshold_range:
    predicted = (np.array(all_errors) > thresh).astype(int)
    
    tp = np.sum((predicted == 1) & (true_labels == 1))
    fp = np.sum((predicted == 1) & (true_labels == 0))
    tn = np.sum((predicted == 0) & (true_labels == 0))
    fn = np.sum((predicted == 0) & (true_labels == 1))
    
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    tpr_values.append(tpr)
    fpr_values.append(fpr)

axes[1, 1].plot(fpr_values, tpr_values, 'b-', linewidth=2, label='ROC Curve')
axes[1, 1].plot([0, 1], [0, 1], 'r--', linewidth=1, label='Random Classifier')

# Mark performance points
for thresh_name, metrics in performance_results.items():
    if thresh_name in ['percentile_95', 'optimal']:
        cm = metrics['confusion_matrix']
        tp, fp, tn, fn = cm[1][1], cm[0][1], cm[0][0], cm[1][0]
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        axes[1, 1].plot(fpr, tpr, 'o', markersize=8, 
                        label=f'{thresh_name} (F1={metrics["f1_score"]:.2f})')

axes[1, 1].set_xlabel('False Positive Rate')
axes[1, 1].set_ylabel('True Positive Rate')
axes[1, 1].set_title('ROC Curve')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n📋 Summary Statistics:")
print(f"   Baseline error rate (assuming 50% anomalies): 50.0%")
print(f"   Best F1-score achieved: {performance_results[best_threshold]['f1_score']:.1%}")
print(f"   Best accuracy achieved: {performance_results[best_threshold]['accuracy']:.1%}")
print(f"   Error separation factor: {np.mean(anomaly_errors) / np.mean(normal_errors):.2f}x")

# Feature importance for anomaly detection (correlation with labels)
if temporal_features_matrix.size > 0:
    feature_anomaly_correlation = []
    extended_labels = [0] * len(normal_samples) + [1] * len(normal_samples)  # Match temporal features
    
    if len(extended_labels) == temporal_features_matrix.shape[0]:
        for i in range(temporal_features_matrix.shape[1]):
            corr = np.corrcoef(temporal_features_matrix[:, i], extended_labels)[0, 1]
            feature_anomaly_correlation.append(abs(corr))
        
        # Find most predictive features
        top_predictive_indices = np.argsort(feature_anomaly_correlation)[-5:][::-1]
        print(f"\n🔍 Most Predictive Temporal Features for Anomaly Detection:")
        for idx in top_predictive_indices:
            corr = np.corrcoef(temporal_features_matrix[:, idx], extended_labels)[0, 1]
            print(f"   {TEMPORAL_FEATURE_NAMES[idx]}: {abs(corr):.3f}")

## 7. Model Comparison & Ablation Study

In [ ]:
# Compare static vs dynamic mode performance (if both models available)
print("🔄 Comparing Static vs Dynamic Mode Performance...")

mode_comparison = {}

# Test static mode if available
try:
    static_model, static_transitions, static_threshold = load_trained_model_and_threshold(
        dataset=CONFIG['dataset'], 
        mode='static'
    )
    
    # Compute errors with static model
    static_errors = []
    for snapshot_path in test_samples[:20]:  # Sample subset
        X = SnapshotLoader.get_X(snapshot_path)
        E = SnapshotLoader.get_E(snapshot_path)
        error = compute_single_graph_error(X, E, static_model, static_transitions, 'static')
        static_errors.append(error)
    
    mode_comparison['static'] = {
        'mean_error': np.mean(static_errors),
        'std_error': np.std(static_errors),
        'threshold_95': static_threshold['percentile_95_threshold'],
        'errors': static_errors
    }
    print("✅ Static mode evaluation completed")
    
except Exception as e:
    print(f"⚠️ Static mode not available: {e}")
    mode_comparison['static'] = None

# Dynamic mode (already loaded)
dynamic_errors_subset = reconstruction_errors[:20]  # Match sample size
mode_comparison['dynamic'] = {
    'mean_error': np.mean(dynamic_errors_subset),
    'std_error': np.std(dynamic_errors_subset),
    'threshold_95': threshold_vector['percentile_95_threshold'],
    'errors': dynamic_errors_subset
}

# Compare if both available
if mode_comparison['static'] is not None:
    print(f"\n📊 Static vs Dynamic Comparison:")
    print(f"{'Metric':<20} {'Static':<15} {'Dynamic':<15} {'Improvement':<15}")
    print("-" * 70)
    
    static_mean = mode_comparison['static']['mean_error']
    dynamic_mean = mode_comparison['dynamic']['mean_error']
    mean_improvement = (dynamic_mean - static_mean) / static_mean * 100
    
    static_std = mode_comparison['static']['std_error']
    dynamic_std = mode_comparison['dynamic']['std_error']
    std_improvement = (dynamic_std - static_std) / static_std * 100
    
    print(f"{'Mean Error':<20} {static_mean:<15.6f} {dynamic_mean:<15.6f} {mean_improvement:<15.2f}%")
    print(f"{'Std Error':<20} {static_std:<15.6f} {dynamic_std:<15.6f} {std_improvement:<15.2f}%")
    
    # Statistical significance test
    from scipy.stats import ttest_ind
    t_stat, p_value = ttest_ind(mode_comparison['static']['errors'], 
                                mode_comparison['dynamic']['errors'])
    print(f"\n🧪 T-test Results:")
    print(f"   T-statistic: {t_stat:.4f}")
    print(f"   P-value: {p_value:.6f}")
    print(f"   Difference is {'significant' if p_value < 0.05 else 'not significant'} (α=0.05)")

# Baseline comparisons
print(f"\n📈 Baseline Comparisons:")

# Simple statistical baselines
baseline_results = {}

# Z-score based anomaly detection
z_scores = np.abs(stats.zscore(reconstruction_errors))
z_threshold = 2.0  # Standard 2-sigma threshold
z_predictions = z_scores > z_threshold
z_anomaly_rate = np.mean(z_predictions)

baseline_results['z_score'] = {
    'method': 'Z-Score (2σ)',
    'anomaly_rate': z_anomaly_rate,
    'threshold': z_threshold
}

# IQR-based detection
q1 = np.percentile(reconstruction_errors, 25)
q3 = np.percentile(reconstruction_errors, 75)
iqr = q3 - q1
iqr_threshold = q3 + 1.5 * iqr
iqr_predictions = reconstruction_errors > iqr_threshold
iqr_anomaly_rate = np.mean(iqr_predictions)

baseline_results['iqr'] = {
    'method': 'IQR Outlier Detection',
    'anomaly_rate': iqr_anomaly_rate,
    'threshold': iqr_threshold
}

# Isolation Forest baseline (if sklearn available)
try:
    from sklearn.ensemble import IsolationForest
    
    # Reshape for sklearn
    errors_reshaped = reconstruction_errors.reshape(-1, 1)
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    iso_predictions = iso_forest.fit_predict(errors_reshaped)
    iso_anomaly_rate = np.mean(iso_predictions == -1)
    
    baseline_results['isolation_forest'] = {
        'method': 'Isolation Forest',
        'anomaly_rate': iso_anomaly_rate,
        'threshold': 'Adaptive'
    }
except ImportError:
    print("   ⚠️ Isolation Forest not available (sklearn required)")

print(f"{'Method':<25} {'Anomaly Rate':<15} {'Threshold':<15}")
print("-" * 60)
for key, result in baseline_results.items():
    threshold_str = f"{result['threshold']:.4f}" if isinstance(result['threshold'], (int, float)) else result['threshold']
    print(f"{result['method']:<25} {result['anomaly_rate']:<15.3%} {threshold_str:<15}")

# Feature ablation study
print(f"\n🔬 Feature Ablation Study:")
print("Analyzing impact of different feature groups...")

feature_groups = {
    'temporal_basic': [0, 1],  # t_norm, seq_position
    'edge_dynamics': [2, 3, 4, 5],  # birth, death, stability, intermittency
    'structural_evolution': [6, 7, 8, 9, 10, 11],  # degree, density, neighborhood, clustering, triangles
    'temporal_patterns': [12, 13],  # autocorr, periodicity
    'centrality': [14, 15, 16],  # centrality evolution and persistence
    'predictability': [17, 18, 19],  # entropy, surprise, edge predictability
    'global_metrics': [20, 21]  # volatility, trend
}

if temporal_features_matrix.size > 0:
    print(f"Feature group importance (variance contribution):")
    for group_name, indices in feature_groups.items():
        if max(indices) < temporal_features_matrix.shape[1]:
            group_variance = np.mean([feature_stats.iloc[i]['Variance'] for i in indices])
            print(f"   {group_name:<20}: {group_variance:.6f}")
        else:
            print(f"   {group_name:<20}: Not available")

## 8. Final Summary & Recommendations

In [ ]:
# Generate comprehensive evaluation report
print("📋 COMPREHENSIVE EVALUATION REPORT")
print("=" * 60)

# Model Performance Summary
print(f"\n🎯 MODEL PERFORMANCE SUMMARY:")
print(f"   Dataset: {CONFIG['dataset']}")
print(f"   Mode: {CONFIG['mode']}")
print(f"   Test samples evaluated: {len(reconstruction_errors)}")
print(f"   Mean reconstruction error: {np.mean(reconstruction_errors):.6f}")
print(f"   Error standard deviation: {np.std(reconstruction_errors):.6f}")

# Threshold Performance
best_f1 = max(performance_results.values(), key=lambda x: x['f1_score'])['f1_score']
print(f"\n🎯 THRESHOLD PERFORMANCE:")
print(f"   Best F1-score achieved: {best_f1:.3f}")
print(f"   Best threshold strategy: {best_threshold}")
print(f"   Recommended threshold: {performance_results[best_threshold]['threshold']:.6f}")

# Feature Analysis
if temporal_features_matrix.size > 0:
    most_important_features = np.argsort(feature_stats['Variance'].values)[-5:][::-1]
    print(f"\n🔍 TEMPORAL FEATURE ANALYSIS:")
    print(f"   Total features analyzed: {temporal_features_matrix.shape[1]}")
    print(f"   Most important features:")
    for i, idx in enumerate(most_important_features, 1):
        print(f"     {i}. {TEMPORAL_FEATURE_NAMES[idx]} (variance: {feature_stats.iloc[idx]['Variance']:.4f})")

# Model Comparison
if mode_comparison['static'] is not None:
    improvement = ((mode_comparison['dynamic']['mean_error'] - mode_comparison['static']['mean_error']) 
                   / mode_comparison['static']['mean_error'] * 100)
    print(f"\n📈 MODEL COMPARISON:")
    print(f"   Dynamic vs Static improvement: {improvement:.2f}%")
    print(f"   Statistical significance: {'Yes' if 'p_value' in locals() and p_value < 0.05 else 'Not tested'}")

# Recommendations
print(f"\n💡 RECOMMENDATIONS:")

# Threshold recommendations
if best_f1 < 0.8:
    print(f"   ⚠️  F1-score ({best_f1:.3f}) suggests room for improvement")
    print(f"       Consider: More training data, feature engineering, or hyperparameter tuning")

if np.std(reconstruction_errors) / np.mean(reconstruction_errors) > 0.5:
    print(f"   ⚠️  High error variance suggests inconsistent performance")
    print(f"       Consider: Data preprocessing, outlier handling, or ensemble methods")

# Feature recommendations
if temporal_features_matrix.size > 0:
    high_corr_count = len([pair for pair in high_corr_pairs if abs(pair[2]) > 0.9])
    if high_corr_count > 3:
        print(f"   🔧 High feature correlation detected ({high_corr_count} pairs > 0.9)")
        print(f"       Consider: Feature selection or dimensionality reduction")

print(f"\n✅ EVALUATION COMPLETED")
print(f"   Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Save evaluation results
evaluation_results = {
    'timestamp': datetime.now().isoformat(),
    'config': CONFIG,
    'reconstruction_error_stats': {
        'mean': float(np.mean(reconstruction_errors)),
        'std': float(np.std(reconstruction_errors)),
        'min': float(np.min(reconstruction_errors)),
        'max': float(np.max(reconstruction_errors))
    },
    'best_threshold': {
        'name': best_threshold,
        'value': float(performance_results[best_threshold]['threshold']),
        'f1_score': float(performance_results[best_threshold]['f1_score'])
    },
    'performance_metrics': {k: {key: float(val) if isinstance(val, (int, float, np.number)) else val 
                                for key, val in v.items() if key != 'confusion_matrix'} 
                            for k, v in performance_results.items()},
    'sample_count': len(reconstruction_errors)
}

# Save to file
results_path = f"evaluation_results_{CONFIG['dataset']}_{CONFIG['mode']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
try:
    with open(results_path, 'w') as f:
        json.dump(evaluation_results, f, indent=2)
    print(f"📁 Results saved to: {results_path}")
except Exception as e:
    print(f"⚠️ Could not save results: {e}")

# Final visualization - Executive Summary Dashboard
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('DiGress Anomaly Detection - Executive Summary Dashboard', fontsize=16, fontweight='bold')

# 1. Error distribution with threshold
axes[0, 0].hist(reconstruction_errors, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(performance_results[best_threshold]['threshold'], color='red', 
                   linestyle='--', linewidth=2, label=f'Best Threshold')
axes[0, 0].set_title('Reconstruction Error Distribution')
axes[0, 0].set_xlabel('Error Value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Performance metrics radar chart (simplified bar chart)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Specificity']
values = [performance_results[best_threshold][m.lower().replace('-', '_')] for m in metrics]
bars = axes[0, 1].bar(metrics, values, color=['#FF9999', '#66B2FF', '#99FF99', '#FFD700', '#FF99CC'])
axes[0, 1].set_title(f'Performance Metrics ({best_threshold})')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_ylim(0, 1)
for bar, value in zip(bars, values):
    axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{value:.2f}', ha='center', va='bottom')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# 3. Feature importance (top 8)
if temporal_features_matrix.size > 0:
    top_8_indices = np.argsort(feature_stats['Variance'].values)[-8:]
    top_8_names = [TEMPORAL_FEATURE_NAMES[i] for i in top_8_indices]
    top_8_values = [feature_stats.iloc[i]['Variance'] for i in top_8_indices]
    
    axes[0, 2].barh(range(len(top_8_names)), top_8_values, color='lightcoral')
    axes[0, 2].set_yticks(range(len(top_8_names)))
    axes[0, 2].set_yticklabels(top_8_names, fontsize=8)
    axes[0, 2].set_title('Top Temporal Features (by Variance)')
    axes[0, 2].set_xlabel('Variance')
    axes[0, 2].grid(True, alpha=0.3)

# 4. Threshold comparison
threshold_names = list(performance_results.keys())
f1_scores = [performance_results[t]['f1_score'] for t in threshold_names]
axes[1, 0].bar(threshold_names, f1_scores, color='lightgreen')
axes[1, 0].set_title('F1-Score by Threshold Strategy')
axes[1, 0].set_ylabel('F1-Score')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# 5. Model comparison (if available)
if mode_comparison['static'] is not None:
    modes = ['Static', 'Dynamic']
    mean_errors = [mode_comparison['static']['mean_error'], mode_comparison['dynamic']['mean_error']]
    axes[1, 1].bar(modes, mean_errors, color=['orange', 'purple'])
    axes[1, 1].set_title('Mean Reconstruction Error by Mode')
    axes[1, 1].set_ylabel('Mean Error')
    for i, v in enumerate(mean_errors):
        axes[1, 1].text(i, v + max(mean_errors)*0.01, f'{v:.4f}', ha='center', va='bottom')
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Static Mode\nNot Available', ha='center', va='center', 
                    transform=axes[1, 1].transAxes, fontsize=12)
    axes[1, 1].set_title('Mode Comparison')

# 6. Summary statistics
summary_text = f"""
Model Performance Summary

Dataset: {CONFIG['dataset']}
Mode: {CONFIG['mode']}
Samples: {len(reconstruction_errors)}

Best Threshold: {best_threshold}
Best F1-Score: {best_f1:.3f}

Error Statistics:
  Mean: {np.mean(reconstruction_errors):.4f}
  Std:  {np.std(reconstruction_errors):.4f}
  
Evaluation Date: {datetime.now().strftime('%Y-%m-%d')}
"""

axes[1, 2].text(0.05, 0.95, summary_text, transform=axes[1, 2].transAxes, 
                fontsize=10, verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
axes[1, 2].set_xlim(0, 1)
axes[1, 2].set_ylim(0, 1)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print(f"\n🎉 Evaluation framework completed successfully!")
print(f"💾 All results have been computed and visualized.")
print(f"📊 Dashboard provides executive-level overview of model performance.")